In [1]:
import IPython, abc, types, sys, importlib.machinery, types, abc, string, jinja2, stringcase
ip = IPython.get_ipython()

A `Phrase` is a `types.ModuleType` the represents the state of a string typically expressed in natural language.  A `Phrase` has no inheritence, but may maintain state. 

* The phrase is converted to a sentence.
* Installed as a `sys.modules` key.
* Installed into __main__ so that the phrases appear with tab completion.

In [2]:
class Phrase(types.ModuleType):    
    def __init__(self, name, doc=None, **dict):
        super().__init__(name, doc)
        # Assign the phrase to main so it recieves valid code prediction.
        setattr(__import__('__main__'), self.__name__, self)
        self.__dict__.update(dict)  

`types.ModuleType` was chosen because it provides an isolated namespace and the import system is a canonical way to access `Phrase` strings.

A `Phrase` has a custom `__call__` method that is syntactic sugar for updating the state of a module.

In [3]:
def __call__(self, doc=None, **objects):         
    """Set the docstring or update the module attributes
    >>> Phrase()"""
    doc and objects.update(__doc__=doc)
    return self.__dict__.update(objects) or self
Phrase.__call__ = __call__

For extra flare, lets use the rshift operator to update the docstring.

In [4]:
def __rshift__(self, object):
    if isinstance(object, dict): self.__dict__.update(object)
    if isinstance(object, str): self.__doc__ = object
    return self

Phrase.__rshift__ = __rshift__

The `Phrase.__name__` is the string representation which is generally a natural language string.

In [5]:
Phrase.__str__ = lambda self: self.__name__

The `repr` provides a deeper representation and fallbacks to `Phrase.__name__`.

In [6]:
Phrase.__repr__ = lambda self: self.__doc__ or self.__name__

Phrase.__doc__ and Phrase.__name__ supply an interactive markdown repr for easy reuse.

Phrase._repr_markdown_ = lambda self: self.__doc__ or f"{str(self)}"

There should be a means to combine `Phrases`.  _At this point, Phrases need to get used._

    Phrase.__and__ = lambda self, object: Phrases((self, object))

    class Phrases(__import__('collections').UserList):
        def __str__(self): return ''.join(
            ('' if data is self[0] else 
             ', ' if data is not self[-1] else
             ' and ') + str(data) for data in self)

        @property
        def __doc__(self): return '\n\n---\n\n'.join(object._repr_markdown_() for object in self)

        _repr_markdown_ = Phrase._repr_markdown_


In [7]:
def to_sentence(string):
    lower, string = string[0].islower(), stringcase.sentencecase(string)
    return [str.upper, str.lower][lower](string[0]) + string[1:]

## Importing phrases.

`Phrases` are __defined by a string that is has different views.__.  I ❤️ hacking the import system.  For `Phrase`s  we will rely on the `__import__` function to load phrases as strings.

    >>> with phrases:
    ...      __import__('This is a phrase') >> "This is a fancy view."

`PhrasesBase` is a stateless contextmanager to "discover" a `importlib.machinery.ModuleSpec` the will become a `Phrase` module.

In [8]:
class PhrasesBase(abc.ABCMeta):
    def __enter__(cls): sys.meta_path.append(phrases)
    def __exit__(cls, *x): sys.meta_path = [x for x in sys.meta_path if not issubclass(x, phrases)]
    def find_spec(cls, name, *args, **kwargs):
        return importlib.machinery.ModuleSpec(name, phrases, is_package=True)

`phrases` is a contextmanager to `__import__` the `Phrase`.

In [9]:
class phrases(metaclass=PhrasesBase):
    @staticmethod
    def create_module(spec):
        return Phrase(spec.name)
    exec_module = staticmethod(lambda module: module)
    is_package = staticmethod(lambda _: True)

In [10]:
import importlib

In [11]:
def phrase(str, doc=None, **globals):
    with phrases:
        return importlib.import_module(str)(doc)(**globals)

In [12]:
def load_ipython_extension(ip):
    ip.display_formatter.formatters['text/markdown'].for_type(
        Phrase, lambda x: repr(x)
    )

In [13]:
def test_importing_with_ipython_syntax():
    with phrases:
        ;__import__ This experiment
        ;str was
        ;__import__ a success
        ;__import__ https://en.wikipedia.org/wiki/Linear_elasticity

Templating should be added.

In [14]:
{
    '{{}}': jinja2.Template,
    '{}': str.format,
    '%': str.__mod__,
    '$': string.Template
}

{'{{}}': jinja2.environment.Template,
 '{}': <method 'format' of 'str' objects>,
 '%': <slot wrapper '__mod__' of 'str' objects>,
 '$': string.Template}